In [35]:
%cd C:/Users/gioc4/Documents/python-benchmark

C:\Users\gioc4\Documents\python-benchmark


In [36]:
df = pd.read_csv("C:/Users/gioc4/Documents/blog/data/sparcs2.zip")

C:\Users\gioc4\AppData\Local\Temp\ipykernel_6236\1063418669.py:1: DtypeWarning: Columns (10,29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("C:/Users/gioc4/Documents/blog/data/sparcs2.zip")


In [37]:
import pandas as pd
from provider_scorecard.benchmark import Benchmark

# load data
df = pd.read_csv("C:/Users/gioc4/Documents/blog/data/sparcs2.zip")
 
# transform
# get only the top 30 drg codes
df = df[df['Hospital County'] == 'Manhattan']

# some recoding
df['Length of Stay'] = [120 if x == '120 +' else x for x in df['Length of Stay']]
df['Length of Stay'] = df['Length of Stay'].astype(int)
df['APR DRG Code'] = df['APR DRG Code'].astype(str)

# top 30 drg codes
drg = df.groupby('APR DRG Code')['APR DRG Code'].count().sort_values(ascending=False).head(100)
drg = drg.astype(str)

# final dataframe
df = df[df['APR DRG Code'].isin(drg.index)]

C:\Users\gioc4\AppData\Local\Temp\ipykernel_6236\1311905022.py:5: DtypeWarning: Columns (10,29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("C:/Users/gioc4/Documents/blog/data/sparcs2.zip")


In [38]:
df.groupby('Facility Name')['Facility Name'].count().sort_values(ascending=False)

Facility Name
Mount Sinai Hospital                                              39074
NYU Langone Hospitals                                             33470
New York-Presbyterian Hospital - New York Weill Cornell Center    32512
New York-Presbyterian Hospital - Columbia Presbyterian Center     30927
Lenox Hill Hospital                                               25761
Bellevue Hospital Center                                          18669
Mount Sinai West                                                  18231
Memorial Hospital for Cancer and Allied Diseases                  15027
Mount Sinai Morningside                                           11734
Hospital for Special Surgery                                      10533
New York Presbyterian Hospital - Allen Hospital                    9882
Mount Sinai Beth Israel                                            8175
New York-Presbyterian/Lower Manhattan Hospital                     7721
Harlem Hospital Center                            

In [39]:
# input features
xfeat = [
    "Age Group",
    "Gender",
    "Race",
    "Length of Stay",
    "Type of Admission",
    "APR DRG Code",
    "APR Severity of Illness Description",
    "APR Risk of Mortality",
]

# evaluation features
efeat = ["Total Charges", "Total Costs", "Payment Typology 1"]

# focal hospitals
hosp = [
    'Mount Sinai Hospital',
    'NYU Langone Hospitals',
    'New York-Presbyterian Hospital - New York Weill Cornell Center',
    'New York-Presbyterian Hospital - Columbia Presbyterian Center',
    'Lenox Hill Hospital'
]

In [44]:
res_list = []
mod_list = []

# loop through hospitals, store dataframes in list
for h in hosp:
    tr = df["Facility Name"] == h

    bch = Benchmark(
        data=df, focal_indicator=tr, predictor_features=xfeat, evaluation_features=efeat
    )
    bch.fit(lrate=0.1, nest = 500, **{'tree_method':'hist', 'alpha': 0, 'lambda': 1})
    bch.evaluate()

    # store results
    mod_list.append(bch)
    res_list.append(
        pd.DataFrame(
            {"Hospital": h, "Outcome": bch.Xeval.columns, "Value": bch.outcomes}
        )
    )

c:\Users\gioc4\Anaconda3\envs\benchmark\Lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
c:\Users\gioc4\Anaconda3\envs\benchmark\Lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
c:\Users\gioc4\Anaconda3\envs\benchmark\Lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
c:\Users\gioc4\Anaconda3\envs\benchmark\Lib\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return is

In [45]:
# concat into single pandas dataframe
results = pd.concat(res_list, axis=0)

In [48]:
mod_list[0].model

XGBClassifier(alpha=0, base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, lambda=1, learning_rate=0.3,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=500, n_jobs=None, ...)

In [46]:
mod_list[1].calc_balance()

Length of Stay:(4.18, 4.03)
Age Group_0 to 17:(0.21, 0.21)
Age Group_18 to 29:(0.09, 0.09)
Age Group_30 to 49:(0.23, 0.23)
Age Group_50 to 69:(0.22, 0.22)
Age Group_70 or Older:(0.26, 0.25)
Gender_F:(0.57, 0.57)
Gender_M:(0.43, 0.43)
Gender_U:(0.0, 0.0)
Race_Black/African American:(0.12, 0.12)
Race_Multi-racial:(0.02, 0.02)
Race_Other Race:(0.28, 0.28)
Race_White:(0.59, 0.59)
Type of Admission_Elective:(0.34, 0.34)
Type of Admission_Emergency:(0.49, 0.49)
Type of Admission_Newborn:(0.14, 0.15)
Type of Admission_Not Available:(0.0, 0.0)
Type of Admission_Trauma:(0.0, 0.0)
Type of Admission_Urgent:(0.03, 0.03)
APR DRG Code_120:(0.01, 0.01)
APR DRG Code_121:(0.01, 0.01)
APR DRG Code_130:(0.0, 0.0)
APR DRG Code_133:(0.01, 0.01)
APR DRG Code_134:(0.0, 0.01)
APR DRG Code_136:(0.0, 0.0)
APR DRG Code_137:(0.03, 0.03)
APR DRG Code_138:(0.01, 0.01)
APR DRG Code_139:(0.01, 0.01)
APR DRG Code_140:(0.01, 0.01)
APR DRG Code_141:(0.0, 0.0)
APR DRG Code_143:(0.0, 0.0)
APR DRG Code_163:(0.01, 0.01)
APR

In [43]:
res_list[0]

,Hospital,Outcome,Value
0,Mount Sinai Hospital,Total Charges,-15131.69
1,Mount Sinai Hospital,Total Costs,-9290.25
2,Mount Sinai Hospital,Payment Typology 1_Blue Cross/Blue Shield,-0.01
3,Mount Sinai Hospital,Payment Typology 1_Federal/State/Local/VA,-0.00
4,Mount Sinai Hospital,"Payment Typology 1_Managed Care, Unspecified",0.01
5,Mount Sinai Hospital,Payment Typology 1_Medicaid,0.00
6,Mount Sinai Hospital,Payment Typology 1_Medicare,0.01
7,Mount Sinai Hospital,Payment Typology 1_Miscellaneous/Other,-0.00
8,Mount Sinai Hospital,Payment Typology 1_Private Health Insurance,-0.02
9,Mount Sinai Hospital,Payment Typology 1_Self-Pay,0.00
